In [1]:
import os
import pandas as pd
import numpy as np

import statsmodels.api as sm
import statsmodels.formula.api as smf

#Pandas settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
#change directory for your path
os.chdir('E:\\OneDrive\\Documents\\GitHub\\eHealthEquity\\Research\\Data Source')

In [23]:
df = pd.read_csv('acs5y_2021_brfss_2021_merged.csv')

index_list = [
    'state','county'
]
brfss_list = [
    col for col in df.columns if col.startswith('brfss_') 
    and not col.endswith('crdprv') and not col.endswith('upper') 
    and not col.endswith('lower')
]
acs_list = [
    col for col in df.columns if col.startswith('pct_')
]

all_columns = index_list + brfss_list + acs_list

df = df[all_columns]

# Feature Engineering

,state,county,brfss_access2_ageadjprv,brfss_checkup_ageadjprv,brfss_csmoking_ageadjprv,brfss_depression_ageadjprv,brfss_diabetes_ageadjprv,brfss_ghlth_ageadjprv,brfss_mhlth_ageadjprv,brfss_obesity_ageadjprv,pct_ed_lt9,pct_ed_9_12,pct_ed_hs,pct_ed_sc,pct_ed_asc,pct_ed_b,pct_ed_gr,pct_age_gte65,pct_race_white,pct_race_black,pct_race_aian,pct_race_asian,pct_race_nhopi,pct_race_other,pct_eth_hisp,pct_sex_male,pct_sex_female,pct_occ_unemp,pct_occ_mgt,pct_occ_svc,pct_occ_sales,pct_occ_nat_res,pct_occ_prod,pct_hlth_unins,pct_ses_pov,pct_tp_veh_0,pct_tp_veh_1,pct_tp_veh_2,pct_tp_veh_3,pct_dg_bb_int,edu_bach
0,Alaska,Aleutians East,16.9,60.7,18.6,13.1,12.6,19.1,12.9,31.3,6.1,9.2,41.6,21.9,5.4,10.6,5.4,11.7,14.6,2.1,44.1,20.9,0.3,6.5,10.5,59.1,40.9,3.5,23.9,13.2,11.7,12.7,38.6,40.0,13.4,24.0,37.2,28.0,10.8,57.5,16.0


# Linear Regression Assumptions Check
1. Linearity between the dependent variable and the independent variables
2. No or little multicollinearity between the independent variables
3. No auto-correlation in the data (i.e. the data is not time-series data)
4. Homoscedasticity (i.e. the variance of the error terms is similar across all the independent variables)
5. Residuals are normally distributed
6. No endogeneity (i.e. the independent variables are not correlated with the error terms)

Linear regression is suseptible to outliers and multicollinearity. 
Multicollinearity is when two or more independent variables are highly correlated with one another.
Multicollinearity can be detected by looking at the correlation matrix or by using VIF (Variance Inflation Factor) scores.
VIF scores of 1.5 to 2.5 are generally considered okay, while scores greater than 10 represent an issue that needs to be addressed.

- Ensure that the independent variables are not correlated with the error terms
- Check for outliers
- Check for normality of the residuals
- Check for homoscedasticity

In [8]:
dv = [
    'brfss_diabetes_ageadjprv'
]
iv = [
    'pct_dg_bb_int'
]
covariates = [
    "brfss_access2_ageadjprv",
    "brfss_checkup_ageadjprv",
    "brfss_csmoking_ageadjprv",
    "brfss_depression_ageadjprv",
    "brfss_mhlth_ageadjprv",
    "brfss_obesity_ageadjprv",
    "pct_ed_lt9",
    "pct_ed_9_12",
    "pct_ed_hs",
    "pct_ed_sc",
    "pct_ed_asc",
    "pct_ed_b",
    "pct_ed_gr",
    "pct_age_gte65",
    "pct_occ_unemp",
    "pct_occ_mgt",
    "pct_occ_svc",
    "pct_occ_sales",
    "pct_occ_nat_res",
    "pct_occ_prod",
    "pct_hlth_unins",
    "pct_ses_pov",
    "pct_tp_veh_0",
    "pct_tp_veh_1",
    "pct_tp_veh_2",
    "pct_tp_veh_3"
]

In [14]:
X = df[iv + covariates]
y = df[dv]
#add constant
X = sm.add_constant(X)

In [26]:
# Simple Multivariate Regression Model
model = sm.OLS(y, X).fit()
model.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                       Results: Ordinary least squares
=============================================================================
Model:               OLS                       Adj. R-squared:      0.921    
Dependent Variable:  brfss_diabetes_ageadjprv  AIC:                 5953.5961
Date:                2023-09-16 05:45          BIC:                 6122.4749
No. Observations:    3076                      Log-Likelihood:      -2948.8  
Df Model:            27                        F-statistic:         1336.    
Df Residuals:        3048                      Prob (F-statistic):  0.00     
R-squared:           0.922                     Scale:               0.40195  
-----------------------------------------------------------------------------
                            Coef.  Std.Err.    t     P>|t|   [0.025   0.975] 
-----------------------------------------------------------------------------
const                      43.7139  30.7552   1.4214 0.1553 -16.5891 104.0170
pct_dg_bb_int              -0.0188   0.0024  -7.6988 0.0000  -0.0236  -0.0140
brfss_access2_ageadjprv     0.1013   0.0042  24.0779 0.0000   0.0931   0.1095
brfss_checkup_ageadjprv     0.0624   0.0035  17.7686 0.0000   0.0555   0.0692
brfss_csmoking_ageadjprv   -0.0070   0.0082  -0.8611 0.3893  -0.0231   0.0090
brfss_depression_ageadjprv -0.1494   0.0069 -21.7570 0.0000  -0.1629  -0.1359
brfss_mhlth_ageadjprv       0.3383   0.0163  20.7911 0.0000   0.3064   0.3702
brfss_obesity_ageadjprv     0.1029   0.0044  23.4859 0.0000   0.0943   0.1115
pct_ed_lt9                 -0.0560   0.1496  -0.3743 0.7082  -0.3493   0.2373
pct_ed_9_12                -0.0225   0.1498  -0.1502 0.8806  -0.3163   0.2712
pct_ed_hs                  -0.1100   0.1496  -0.7354 0.4621  -0.4033   0.1833
pct_ed_sc                  -0.0871   0.1495  -0.5827 0.5601  -0.3804   0.2061
pct_ed_asc                 -0.1063   0.1496  -0.7101 0.4777  -0.3997   0.1872
pct_ed_b                   -0.1029   0.1496  -0.6878 0.4916  -0.3962   0.1904
pct_ed_gr                  -0.0683   0.1497  -0.4566 0.6480  -0.3618   0.2251
pct_age_gte65              -0.0348   0.0031 -11.2137 0.0000  -0.0409  -0.0287
pct_occ_unemp               0.0934   0.0102   9.1242 0.0000   0.0733   0.1135
pct_occ_mgt                -0.2532   0.1773  -1.4280 0.1534  -0.6008   0.0945
pct_occ_svc                -0.2397   0.1774  -1.3513 0.1767  -0.5875   0.1081
pct_occ_sales              -0.2554   0.1773  -1.4406 0.1498  -0.6030   0.0922
pct_occ_nat_res            -0.2723   0.1774  -1.5356 0.1247  -0.6201   0.0754
pct_occ_prod               -0.2540   0.1772  -1.4330 0.1520  -0.6015   0.0935
pct_hlth_unins             -0.0037   0.0036  -1.0471 0.2951  -0.0107   0.0032
pct_ses_pov                 0.0929   0.0039  23.5948 0.0000   0.0852   0.1006
pct_tp_veh_0               -0.0897   0.1980  -0.4529 0.6507  -0.4780   0.2986
pct_tp_veh_1               -0.0909   0.1980  -0.4591 0.6462  -0.4791   0.2973
pct_tp_veh_2               -0.1103   0.1980  -0.5570 0.5776  -0.4986   0.2780
pct_tp_veh_3               -0.0937   0.1980  -0.4730 0.6363  -0.4819   0.2946
-----------------------------------------------------------------------------
Omnibus:                  107.158          Durbin-Watson:             1.284  
Prob(Omnibus):            0.000            Jarque-Bera (JB):          130.446
Skew:                     0.404            Prob(JB):                  0.000  
Kurtosis:                 3.604            Condition No.:             403528 
=============================================================================
Notes:
[1] Standard Errors assume that the covariance matrix of the errors is
correctly specified.
[2] The condition number is large, 4.04e+05. This might indicate
that there are strong multicollinearity or other numerical
problems.
"""

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices